# Introduction/Business Problem

**In Hamburg, if someone is looking to open a restaurant, where would you recommend that they open it? Similarly, if a contractor is trying to start their own hotel business, where would you recommend that they setup their hotel?**

# Used Data

**We will use Data from Wikipedia to find the Postal Codes for each Neighborhood, the Foursquare API and the Geospatial data (https://cocl.us/Geospatial_data) to find the Latitude and Longitude for each Neighborhood.**

# Methology

# Results

# Discussion

# Conclusion

In [193]:
# Load and aggregate data
# load packages
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import urllib.request
from bs4 import BeautifulSoup

import requests
# load data
all_df = pd.read_csv("https://launix.de/launix/wp-content/uploads/2019/06/PLZ.csv", sep = ";", header = None, names=['PostalCode', 'City', 'Longitude', 'Latitude'])



In [194]:
# filter hamburg_df 
hamburg_df = all_df[all_df['City']=='Hamburg']
hamburg_df.head()

,PostalCode,City,Longitude,Latitude
1228,20095,Hamburg,10.001104,53.554158
1229,20097,Hamburg,10.020163,53.549704
1230,20099,Hamburg,10.011777,53.560801
1231,20144,Hamburg,9.975633,53.577054
1232,20146,Hamburg,9.978171,53.570563


In [195]:
hamburg_df.dtypes

PostalCode      int64
City           object
Longitude     float64
Latitude      float64
dtype: object

In [196]:
# define function venue information
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [207]:
# define Keys
CLIENT_ID = 'S5X0WCZLWFLIGAQ0GU35DJWRSTF0GHUHX0TMXGANR0NPC34U' # your Foursquare ID
CLIENT_SECRET = 'DDK2MRKMUMUDFDJ43RIZCLZMY31Q3YYAU2M1RENNTTAIVMO3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100


# get venue information
hamburg_venues = getNearbyVenues(names=hamburg_df['PostalCode'],
                                   latitudes=hamburg_df['Latitude'],
                                   longitudes=hamburg_df['Longitude']
                                  )



20095
20097
20099
20144
20146
20148
20149
20249
20251
20253
20255
20257
20259
20354
20355
20357
20359
20457
20459
20535
20537
20539
21029
21031
21033
21035
21037
21039
21073
21075
21077
21079
21107
21109
21129
21147
21149
22041
22043
22045
22047
22049
22081
22083
22085
22087
22089
22111
22115
22117
22119
22143
22145
22147
22149
22159
22175
22177
22179
22297
22299
22301
22303
22305
22307
22309
22335
22337
22339
22359
22391
22393
22395
22397
22399
22415
22417
22419
22453
22455
22457
22459
22523
22525
22527
22529
22547
22549
22559
22587
22589
22605
22607
22609
22761
22763
22765
22767
22769
27499


In [208]:
# Inspect df
hamburg_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,20095,53.554158,10.001104,Sakura Sushi,53.552442,10.000058,Sushi Restaurant
1,20095,53.554158,10.001104,Hamburger Kunsthalle - Galerie der Gegenwart,53.555376,10.002739,Art Museum
2,20095,53.554158,10.001104,Barceló Hamburg,53.554432,10.000625,Hotel
3,20095,53.554158,10.001104,Michelle Records,53.552378,10.000935,Record Shop
4,20095,53.554158,10.001104,Quan Do,53.553373,10.002558,Vietnamese Restaurant


In [209]:
hamburg_venues.shape

(1520, 7)

In [210]:
# Filter on PostalCodes with more than 10 venues
hamburg_venues_filt = hamburg_venues[hamburg_venues.groupby('Neighborhood')['Neighborhood'].transform('size') > 9].reset_index()

In [211]:
hamburg_venues_filt.shape

(1288, 8)

In [212]:
# analyze neighborhood
hamburg_onehot = pd.get_dummies(hamburg_venues_filt[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hamburg_onehot['Neighborhood'] = hamburg_venues_filt['Neighborhood']

# get list of columns
cols = list(hamburg_onehot)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Neighborhood')))

hamburg_onehot = hamburg_onehot[cols]

hamburg_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Service,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Automotive Shop,BBQ Joint,Baby Store,Bakery,Bank,Bar,Bavarian Restaurant,Bed & Breakfast,Beer Bar,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Stop,Business Service,Cafeteria,Café,Camera Store,Canal,Candy Store,Cantonese Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Drugstore,Duty-free Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Food & Drink Shop,Food Court,Food Service,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Korean Restaurant,Kumpir Restaurant,Lake,Laser Tag,Laundromat,Leather Goods Store,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Museum,Music School,Music Venue,Night Market,Nightclub,Opera House,Optical Shop,Organic Grocery,Pakistani Restaurant,Palatine Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pier,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swabian Restaurant,Tanning Salon,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Track Stadium,Trail,Train Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Water Park,Whisky Bar,Wine Bar,Wine Shop
0,20095,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20095,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [213]:
# get size
hamburg_onehot.shape

(1288, 226)

In [214]:
# get the means of categories
hamburg_grouped = hamburg_onehot.groupby('Neighborhood').mean().reset_index()
hamburg_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Service,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Automotive Shop,BBQ Joint,Baby Store,Bakery,Bank,Bar,Bavarian Restaurant,Bed & Breakfast,Beer Bar,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Stop,Business Service,Cafeteria,Café,Camera Store,Canal,Candy Store,Cantonese Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Drugstore,Duty-free Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Food & Drink Shop,Food Court,Food Service,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Korean Restaurant,Kumpir Restaurant,Lake,Laser Tag,Laundromat,Leather Goods Store,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Museum,Music School,Music Venue,Night Market,Nightclub,Opera House,Optical Shop,Organic Grocery,Pakistani Restaurant,Palatine Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pier,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swabian Restaurant,Tanning Salon,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Track Stadium,Trail,Train Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Water Park,Whisky Bar,Wine Bar,Wine Shop
0,20095,0.0,0.000000,0.0,0.0,0.01,0.0,0.0,0.0,0.01,0.0,0.020000,0.00000,0.01,0.0,0.01,0.0,0.020000,0.000000,0.020000,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.01,0.0,0.0,0.0,0.030000,0.0,0.0,0.0,0.0,0.000000,0.0,0.01,0.030000,0.000000,0.0,0.0,0.030000,0.0,0.000000,0.02,0.01,0.0,0.0,0.02,0.0,0.0,0.0,0.02000,0.01,0.0,0.040000,0.0,0.0,0.0,0.02,0.0,0.000000,0.0,0.0,0.000000,0.01,0.000000,0.0,0.0,0.000000,0.01,0.020000,0.0,0.0,0.010000,0.0,0.010000,0.000000,0.010000,0.0,0.000000,0.0,0.0,0.0,0.01,0.000000,0.0,0.01,0.0,0.020000,0.0,0.0,0.0,0.030000,0.01,0.01,0.000000,0.01,0.0,0.010000,0.0,0.000000,0.0,0.010000,0.000000,0.050000,0.01000,0.0,0.000000,0.000000,0.010000,0.0,0.000000,0.0,0.0,0.01,0.0,0.0,0.010000,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.000000,0.000000,0.020000,0.0,0.010000,0.010000,0.0,0.0,0.0,0.01,0.0,0.000000

In [215]:
# function for sorting venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [220]:
# display top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = hamburg_grouped['Neighborhood']

for ind in np.arange(hamburg_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hamburg_grouped.iloc[ind, :], num_top_venues)

    # add clustering labels
    
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,20095,Hotel,Restaurant,Coffee Shop,Burger Joint,Café,Bookstore,Theater,German Restaurant,Bakery,Furniture / Home Store
1,20097,Bakery,Hotel,Vegetarian / Vegan Restaurant,Italian Restaurant,Coffee Shop,Food & Drink Shop,Fast Food Restaurant,Pizza Place,Soup Place,Sandwich Place
2,20099,Hotel,Italian Restaurant,French Restaurant,Café,Park,Restaurant,German Restaurant,Burger Joint,Clothing Store,Coffee Shop
3,20144,Steakhouse,Café,Drugstore,Electronics Store,Park,Spanish Restaurant,Sporting Goods Shop,Organic Grocery,Supermarket,Sushi Restaurant
4,20146,Café,Bakery,Italian Restaurant,Ice Cream Shop,Indian Restaurant,Drugstore,Diner,Restaurant,Salon / Barbershop,Bookstore


In [221]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# run cluster algorithm
# set number of clusters
kclusters = 5

hamburg_grouped_clustering = hamburg_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hamburg_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
kmeans.labels_

array([1, 1, 1, 3, 3, 3, 1, 3, 3, 1, 3, 3, 3, 1, 1, 1, 1, 1, 3, 4, 1, 3,
       3, 3, 3, 2, 2, 3, 3, 3, 3, 0, 2, 3, 3, 3])

In [222]:
# create new df that includes cluster
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
hamburg_merged = hamburg_df

# merge hamburg_grouped with hamburg_data to add latitude/longitude for each neighborhood
hamburg_merged = hamburg_merged.merge(neighborhoods_venues_sorted.set_index('Neighborhood'), how = 'left', left_on = 'PostalCode',right_on='Neighborhood')

# filter only districts where we have data
hamburg_merged = hamburg_merged[hamburg_merged["Cluster Labels"].notnull()]

In [223]:
hamburg_merged.dtypes

PostalCode                  int64
City                       object
Longitude                 float64
Latitude                  float64
Cluster Labels            float64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [224]:
hamburg_merged['PostalCode'] = hamburg_merged['PostalCode'].apply(str)
hamburg_merged['Cluster Labels'] = hamburg_merged['Cluster Labels'].astype(np.int64)

In [225]:
hamburg_merged.shape

(36, 15)

In [226]:

latitude = 53.551086
longitude = 9.993682

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import folium
import folium
# visualize clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)



In [227]:
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hamburg_merged['Latitude'], hamburg_merged['Longitude'], hamburg_merged['PostalCode'], hamburg_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [234]:
# Inspect Clusters
# Cluster 0
hamburg_merged[hamburg_merged["Cluster Labels"]==0]

,PostalCode,City,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,22335,Hamburg,9.997822,53.631971,0,Snack Place,Airport Service,Lingerie Store,Smoke Shop,Airport,Sushi Restaurant,Electronics Store,Duty-free Shop,Food & Drink Shop,Flea Market


In [235]:
# Inspect Clusters
# Cluster 1
hamburg_merged[hamburg_merged["Cluster Labels"]==1]

,PostalCode,City,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,20095,Hamburg,10.001104,53.554158,1,Hotel,Restaurant,Coffee Shop,Burger Joint,Café,Bookstore,Theater,German Restaurant,Bakery,Furniture / Home Store
1,20097,Hamburg,10.020163,53.549704,1,Bakery,Hotel,Vegetarian / Vegan Restaurant,Italian Restaurant,Coffee Shop,Food & Drink Shop,Fast Food Restaurant,Pizza Place,Soup Place,Sandwich Place
2,20099,Hamburg,10.011777,53.560801,1,Hotel,Italian Restaurant,French Restaurant,Café,Park,Restaurant,German Restaurant,Burger Joint,Clothing Store,Coffee Shop
6,20149,Hamburg,9.992131,53.581092,1,Boat Rental,Café,Hotel,Park,Wine Shop,Seafood Restaurant,Bank,Bar,Bistro,Bus Stop
9,20253,Hamburg,9.966791,53.582910,1,Italian Restaurant,Café,Cocktail Bar,Pizza Place,Furniture / Home Store,Burger Joint,Doner Restaurant,Ice Cream Shop,Vietnamese Restaurant,Gym / Fitness Center
13,20354,Hamburg,9.989227,53.558531,1,Bakery,Café,Hotel,Coffee Shop,Steakhouse,Cocktail Bar,Cosmetics Shop,Italian Restaurant,Botanical Garden,Gym / Fitness Center
14,20355,Hamburg,9.978956,53.558257,1,Hotel,Café,Botanical Garden,Gym / Fitness Center,Nightclub,Garden,Burger Joint,Shoe Store,Concert Hall,Mediterranean Restaurant
15,20357,Hamburg,9.966303,53.565085,1,Restaurant,Bar,Gastropub,Café,Pub,Beer Store,Park,Burger Joint,Steakhouse,Coffee Shop
16,20359,Hamburg,9.965141,53.553306,1,Theater,Bar,Café,Hotel,Pub,Plaza,Lounge,Cocktail Bar,Clothing Store,Ice Cream Shop
18,20459,Hamburg,9.976965,53.547883,1,Portuguese Restaurant,Café,Hotel,Seafood Restaurant,Italian Restaurant,German Restaurant,Bakery,Sushi Restaurant,Gastropub,French Restaurant


In [236]:
# Inspect Clusters
# Cluster 2
hamburg_merged[hamburg_merged["Cluster Labels"]==2]

,PostalCode,City,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,22111,Hamburg,10.082665,53.550699,2,Park,Supermarket,Bakery,Post Office,Laundromat,Pharmacy,Café,Bus Stop,Mediterranean Restaurant,Convenience Store
50,22119,Hamburg,10.105736,53.559444,2,Supermarket,Bus Stop,Bakery,Liquor Store,Sandwich Place,Discount Store,Lake,Bank,Shopping Mall,Event Space
85,22529,Hamburg,9.955756,53.598736,2,Supermarket,Italian Restaurant,Pharmacy,Bus Stop,Bakery,Café,Soccer Field,Climbing Gym,Design Studio,Food & Drink Shop


In [237]:
# Inspect Clusters
# Cluster 3
hamburg_merged[hamburg_merged["Cluster Labels"]==3]

,PostalCode,City,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,20144,Hamburg,9.975633,53.577054,3,Steakhouse,Café,Drugstore,Electronics Store,Park,Spanish Restaurant,Sporting Goods Shop,Organic Grocery,Supermarket,Sushi Restaurant
4,20146,Hamburg,9.978171,53.570563,3,Café,Bakery,Italian Restaurant,Ice Cream Shop,Indian Restaurant,Drugstore,Diner,Restaurant,Salon / Barbershop,Bookstore
5,20148,Hamburg,9.996387,53.569405,3,Italian Restaurant,Steakhouse,Café,Restaurant,Bar,Bike Rental / Bike Share,Spa,Chinese Restaurant,Pub,Church
7,20249,Hamburg,9.988463,53.592227,3,Park,Café,Italian Restaurant,Bus Stop,Supermarket,Drugstore,Bakery,Gastropub,Coffee Shop,German Restaurant
8,20251,Hamburg,9.977862,53.595006,3,Bakery,Bistro,Café,Gastropub,Supermarket,Gym / Fitness Center,Bus Stop,Laser Tag,Science Museum,Coffee Shop
10,20255,Hamburg,9.952074,53.581492,3,Café,Bar,Ice Cream Shop,Supermarket,Italian Restaurant,Pharmacy,Grocery Store,Bakery,Asian Restaurant,German Restaurant
11,20257,Hamburg,9.942673,53.579107,3,Café,Bakery,Bar,Ice Cream Shop,Pub,Sushi Restaurant,Asian Restaurant,Falafel Restaurant,Furniture / Home Store,French Restaurant
12,20259,Hamburg,9.957193,53.575022,3,Café,Coffee Shop,Greek Restaurant,Park,Italian Restaurant,Drugstore,Bar,Supermarket,Bakery,Dessert Shop
19,20535,Hamburg,10.049668,53.562062,3,Bakery,Soccer Field,Gastropub,Playground,Park,Restaurant,Drugstore,Salon / Barbershop,Business Service,Bus Stop
42,22081,Hamburg,10.043227,53.578604,3,Supermarket,Bar,BBQ Joint,Bank,Falafel Restaurant,Hotel,Burger Joint,Dessert Shop,Spanish Restaurant,Bus Stop


In [238]:
# Inspect Clusters
# Cluster 4
hamburg_merged[hamburg_merged["Cluster Labels"]==4]

,PostalCode,City,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,20537,Hamburg,10.049056,53.55245,4,Supermarket,Greek Restaurant,Hotel,Rental Car Location,Electronics Store,Chinese Restaurant,German Restaurant,IT Services,Event Space,Eastern European Restaurant


In [255]:
# District with most Restaurants
hamburg_venues[hamburg_venues["Venue Category"].str.contains("Restaurant")].groupby("Neighborhood").size().sort_values(ascending = False)

Neighborhood
20459    43
20095    27
20146    23
20255    21
22299    16
20099    16
20253    16
20354    16
20257    14
20357    14
22765    13
20259    12
20359    12
22087    11
22305    10
20355    10
20148     9
22083     9
22085     9
20249     8
20097     8
22301     7
20537     4
22307     4
20144     4
22041     4
22767     3
22081     3
20535     2
22559     2
22047     2
20149     2
21073     2
22175     2
22045     1
22399     1
22303     1
22763     1
22397     1
22761     1
22297     1
22177     1
22609     1
22587     1
22159     1
22529     1
22523     1
22147     1
22455     1
22419     1
22415     1
22115     1
22111     1
20539     1
21033     1
22335     1
22339     1
dtype: int64

In [272]:
# District with most Hotels
hamburg_venues[hamburg_venues["Venue Category"].str.contains("Hotel")].groupby("Neighborhood").size().sort_values(ascending = False)

Neighborhood
20459    7
20099    6
20095    6
20097    4
20359    4
20354    4
20355    3
22459    2
20149    2
20537    2
22415    1
20144    1
20148    1
20253    1
22305    1
22087    1
22457    1
21149    1
22041    1
22081    1
22083    1
22085    1
20357    1
dtype: int64